In [2]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 加载模型和分词器
model_path = "./Meta-Llama-3.1-8B-Instruct/"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
import re

# 输入 captions
captions = [
    "A group of people are playing basketball on a court.",
    "A man is walking alone in the park.",
    "Two people are shaking hands in an office.",
    "A woman is reading a book by herself.",
    "A tennis player is interviewed by a reporter on a tennis court.",
    "A man standing next to a woman in a dance studio.",
    "A man and a woman are dancing on the sidewalk of a city street.",
    "A man hugging a woman in a room full of chairs.",
    "A man in a blue shirt wrestling another man on a green mat.",
    "Two people are in a room, one is an older man wearing a shirt and the other is a younger man wearing a shirt, they are shaking hands."
]

# 统计结果
results = []

for caption in captions:
    messages = [
        {"role": "system", "content": """
         Determine if the following video caption describes a scene where multiple people are clearly engaged in physical interaction (e.g., shaking hands, hugging, playing sports together, etc.).
         Answer with 'Yes' only if the caption explicitly describes physical interaction involving multiple people. Otherwise, answer with 'No' or 'Unknown'.
         """},
        {"role": "user", "content": caption},
    ]

    # 格式化输入
    input_text = tokenizer.apply_chat_template(messages, tokenize=False)

    # 编码输入并移动到 GPU
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    # 生成输出
    output = model.generate(**inputs, max_new_tokens=50)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # 提取 assistant 的输出
    assistant_output = generated_text.split("assistant")[-1].strip()

    results.append(assistant_output)

# 打印结果
print(results)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes']


: 

In [2]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    temperature=0.1,  # 控制随机性
    top_p=0.9,        # 控制多样性
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0